In [1]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import import_ipynb
from sklearn.model_selection import train_test_split
from pyspark import SparkContext, SparkConf, SQLContext
import os.path

 # Função extrairDados 
# Parametros:
- info: Dicionario com as informações de IP e base.
- consulta: String com a consulta a ser executada em cada base da listIp.
- arquivo: String com nome do arquivo a ser gerado ao csv.
- spark: Boolean com a identificação de geração de um dataframe para Spark (True) ou para Pandas (False).
- objspark: Objeto de contexto do spark.

# Descrição: 
- Função responsavel por gerar a extração dos dados com base numa lista de ips informados, onde com base no codigo passado.  como parametro será executado para cada instancia o codigo sql.

In [4]:
'''
@param info:
@param consulta:
@param arquivo:
@param spark:
@param objspark:
@return:
'''
def extrairDados(info,consulta,path,arquivo = None,dataFrame = True,spark = False,objSpark = None):
    conn_str  = 'DRIVER={SQL Server};SERVER='+info['ip']+';DATABASE='+info['base']+';Integrated Security=true'
    if spark:
        sc = objSpark.sparkContext
        sqlContext = SQLContext(sc)
    
    if not (os.path.exists(path)):
        os.mkdir(path)
    
    try:
        if arquivo == None:
            conn = pyodbc.connect(conn_str)
            cursor = conn.cursor()
            codigo = consulta
            df_tmp = pd.read_sql_query(codigo,conn)
            conn.close()
            if dataFrame:
                if spark:
                    df_tmp = objSpark.createDataFrame(df_tmp)
        if arquivo != None:
            if os.path.exists(path+arquivo+".parquet"):
                    df_tmp = pd.read_parquet(path+arquivo+".parquet")
            else: 
                conn = pyodbc.connect(conn_str)
                cursor = conn.cursor()
                codigo = consulta
                df_tmp = pd.read_sql_query(codigo,conn)
                conn.close()
                export_parquet = df_tmp.to_parquet(r''+path+arquivo+".parquet", engine='auto')
                if dataFrame:
                    if spark:
                        df_tmp = objSpark.createDataFrame(df_tmp)
        return df_tmp
    except Exception as ex:
        print("Erro: "+str(ex))
        return None 

 # Função resultQuery 
# Parametros:
- arquivo: Nome do arquivo sql de consulta


# Descrição: 
- Função responsavel por devolver o conteudo do arquivo .sql

In [1]:
def resultQuery(arquivo):
    arq = open("querys/"+arquivo,"r+")
    sql = arq.read()
    arq.close()
    return sql

### [getresultdataset] - Printa os totais previsto x reais x diferença x porcentagem

In [2]:
def getresultdataset(dt, model_name):
    total_diff = dt.aggregate(['sum'])['diff'][0]
    total_absolute_diff = dt['diff'].apply(lambda c: np.absolute(c).sum()).sum()
    total_real = dt.aggregate(['sum'])['Real'][0]
    total_prev = dt.aggregate(['sum'])['Previsto'][0]
    total_reg  = dt.aggregate(['count'])['Real'][0]
    
    # Percentual de diferença do previsto com o real  
    total_percent_diff = (((dt['Real'] - dt['Previsto']) / dt['Real']) * 100)
    
    # Remove valores infinitivos
    total_percent_diff          = total_percent_diff.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    
    # Obtem total absoluto dos percentuais da diferença
    total_percent_absolute_diff = total_percent_diff.apply(lambda x: np.absolute(x).sum()).sum()
    
    # Obtem total não absoluto dos percentuais da diferença
    total_percent_diff          = total_percent_diff.sum()
    
    print("SE - Sum of Error: ")
    print("["+model_name+"] - Total Real:  %.2f" %(total_real))
    print("["+model_name+"] - Total Prev:  %.2f" %(total_prev))
    print("["+model_name+"] - SE Final:  %.2f" %(total_diff))
    print("["+model_name+"] - Total Porcentagem de diferença do previsto x real:  {:.2f}%".format(np.absolute(((total_diff / total_real)*100))))
    print("----------------------------------------------------------")
    
    print("ME - Mean Error:  ")
    print("["+model_name+"] - Total Registros: %.f" % (total_reg))
    print("["+model_name+"] - Total Diff : %.f" % (total_diff))
    print("["+model_name+"] - ME Final : %.6f" % (total_diff / total_reg))
    print("----------------------------------------------------------")
    
    print("MAE - Mean Absolute Error:  ")
    print("["+model_name+"] - Total Registros: %.f" % (total_reg))
    print("["+model_name+"] - Total Absolute Diff : %.f" % (total_absolute_diff))
    print("["+model_name+"] - ME Final : %.6f" % (total_absolute_diff / total_reg))
    print("----------------------------------------------------------")
    
    print("MPE - Mean Percentage Error:  ")
    print("["+model_name+"] - Total Registros: %.f" % (total_reg))
    print("["+model_name+"] - Total percent Diff: %.f" % (total_percent_diff))
    print("["+model_name+"] - MPE Final : {:.2f}%".format(total_percent_diff / total_reg))
    print("----------------------------------------------------------")
    
    print("MAPAE - Mean Absolute Percentage Error:  ")
    print("["+model_name+"] - Total Registros: %.f" % (total_reg))
    print("["+model_name+"] - Total percent Absolute Diff: %.f" % (total_percent_absolute_diff))
    print("["+model_name+"] - MAPAE Final : {:.2f}%".format(total_percent_absolute_diff / total_reg))

### [transformDataSet] - Transforma dois arrays em colunas Previsto X Real

In [3]:
def transformDataSet(previsto, real):
    d = {"Previsto": [x[0] for x in previsto], "Real":[x[0] for x in real]}
    df_resultado = pd.DataFrame(data=d)
    df_resultado['diff'] = df_resultado.Previsto - df_resultado.Real
    df_resultado['porcen'] =  np.absolute(((df_resultado.Previsto / df_resultado.Real)*100)-100)
    df_resultado = df_resultado.round(2)
    df_resultado = df_resultado.dropna()
    return df_resultado

### [preperData] - Prepara os dados reais para predição. (Dados não de treino).

In [6]:
def preperData(df, preditoras,prevista):
     # Seleção de variáveis preditoras (Feature Selection)
    atributos = preditoras
    # Variável a ser prevista
    atrib_prev = prevista

    # Criando objetos
    X = np.absolute(df[atributos].values)
    Y = df[atrib_prev].values


    Y = Y.astype(int)
    
    return X, Y

### [preperTreinoTeste] - preparada dados de acordo com as preditos x alvos para gerar dados de treino e teste

In [9]:
def preperTreinoTeste(df, preditoras, prevista, split):
    # Seleção de variáveis preditoras (Feature Selection)
    atributos = preditoras
    # Variável a ser prevista
    atrib_prev = prevista

    # Criando objetos
    X = np.absolute(df[atributos].values)
    Y = df[atrib_prev].values


    # Definindo a taxa de split
    split_test_size = split
    # Criando dados de treino e de teste
    X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

    Y_treino = Y_treino.astype(int)
    Y_teste  = Y_teste.astype(int)
    
    return X_treino, Y_treino, X_teste, Y_teste
